In [12]:
import sys

# Davide's epynet
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'scripts')

from scripts import network
from scripts import utils

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

#import wntr
#import wntr.network.controls as controls
import csv
import pandas as pd
import yaml
import time
import datetime

## Davide C-Town epynet simulation

In [ ]:
net = network.WaterDistributionNetwork("ctown.inp")
step = 300
duration = 3600
net.set_time_params(duration=duration, hydraulic_step=step) # duration=24h, hydstep=10min

In [ ]:
status = [1.0, 0.0]
actuators_status_dict = {uid: status for uid in net.pumps.uid.append(net.valves.uid)}

In [ ]:
net.run(interactive=True, status_dict=actuators_status_dict)
no_step_nodes_report = net.df_nodes_report
no_step_links_report = net.df_links_report

# Step-by-step simulation

In [ ]:
net = network.WaterDistributionNetwork("ctown.inp")
step = 300
duration = 3600

net.set_time_params(duration=duration, hydraulic_step=step) # duration=24h, hydstep=10min

status = [1.0, 0.0]
actuators_status_dict = {uid: status for uid in net.pumps.uid.append(net.valves.uid)}

timestep = 1

simulation_time = 0 
sim_values = []

net.init_simulation(interactive=True)

# toDo: We would have to implement a way of having a master_time (iterations)
while timestep > 0:
    
    # get actuator_status from DB
    # create list of dictionaries = actuators_status_dict

    # /Andres: This is how it would look in the future/
    #net.update_actuators(actuators_status_dict)
    #timestep, network_state = net.simulate_step(simulation_time)
    
    timestep, network_state = net.simulate_step(simulation_time, actuators_status_dict)
        
    # update DB with network_state
    
    simulation_time = simulation_time + timestep
    
    print("current timestep: " + str(timestep))    
    print("current simulation_time: " + str(simulation_time))

In [ ]:
net.create_df_reports()
step_nodes_results = net.df_nodes_report
step_links_results = net.df_links_report

In [ ]:
no_step_df_tanks = no_step_nodes_report.iloc[:, no_step_nodes_report.columns.get_level_values(2)=='pressure']['tanks'] 
step_df_tanks = step_nodes_results.iloc[:, step_nodes_results.columns.get_level_values(2)=='pressure']['tanks']

In [ ]:
no_step_df_tanks

In [ ]:
step_df_tanks

In [ ]:
def do_subplot(series, variable_names, variable_type, legends, xlim=None, ylim=None):
    subplot_hspace = 0.6
    subplot_wspace = 0.1
    subplot_y_num = 2
    subplot_x_num = (len(variable_names)/subplot_y_num) + 1    
    i = 0

    
    for variable in variable_names:
        dd = plt.subplot(subplot_x_num,subplot_y_num,i+1)
        j = 0
        for serie in series:                        
            #serie[variable].plot(label=legends[j], ax=dd)        
            dd.plot(serie[variable][variable_type], label=legends[j], alpha=0.75)
            j = j + 1

        plt.title(variable_names[i])
        plt.xlabel('Iterations')
        plt.ylabel('tank level (m)')                       
        if xlim:
            plt.xlim(left=xlim[0], right=xlim[1])                    
        if ylim:
            plt.ylim(left=ylim[0], right=ylim[1])            
            
        plt.legend()
        plt.subplots_adjust(top=2.00, right=3.00, hspace = subplot_hspace, wspace=subplot_wspace)
        plt.grid(True)
        
        i = i+1

In [ ]:
tanks = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7'] 
do_subplot([no_step_df_tanks, step_df_tanks], tanks, 'pressure',['No Step-by-step', 'Step-by-step'])

### Ctown PDA for 24h

In [14]:
net = network.WaterDistributionNetwork("ctown_pd.inp")
step = 300
duration = 86400
net.set_time_params(duration=duration, hydraulic_step=step) # duration=24h, hydstep=5min

In [16]:
net.run()

In [18]:
net.df_links_report

link                  pumps                                                 \
id                      PU1                          PU2                     
properties             flow     energy status       flow     energy status   
0 days 00:00:00   96.590064  43.065440    1.0  96.609061  43.060132    1.0   
0 days 00:05:00   96.599741  43.062737    1.0  96.618741  43.057425    1.0   
0 days 00:10:00   96.609324  43.060059    1.0  96.628326  43.054743    1.0   
0 days 00:15:00   96.618863  43.057391    1.0  96.637868  43.052071    1.0   
0 days 00:20:00   96.628358  43.054734    1.0  96.647365  43.049410    1.0   
...                     ...        ...    ...        ...        ...    ...   
0 days 23:40:32  120.076905  31.490750    1.0   0.000000   0.000000    0.0   
0 days 23:45:00  120.110398  31.466895    1.0   0.000000   0.000000    0.0   
0 days 23:50:00  120.146413  31.441218    1.0   0.000000   0.000000    0.0   
0 days 23:55:00  120.182399  31.415536    1.0   0.000000   0.000000    0.0   
1 days 00:00:00  119.431414  31.946346    1.0   0.000000   0.000000    0.0   

link                                           ... valves                      \
id               PU3                      PU4  ...     v1       V45             
properties      flow energy status       flow  ... status  velocity      flow   
0 days 00:00:00  0.0    0.0    0.0  33.925675  ...    1.0  0.132768  2.421893   
0 days 00:05:00  0.0    0.0    0.0  33.915281  ...    1.0  0.132767  2.421875   
0 days 00:10:00  0.0    0.0    0.0  33.904866  ...    1.0  0.132767  2.421875   
0 days 00:15:00  0.0    0.0    0.0  33.894426  ...    1.0  0.132767  2.421875   
0 days 00:20:00  0.0    0.0    0.0  33.883961  ...    1.0  0.132767  2.421875   
...              ...    ...    ...        ...  ...    ...       ...       ...   
0 days 23:40:32  0.0    0.0    0.0  35.469591  ...    1.0  0.142089  2.591935   
0 days 23:45:00  0.0    0.0    0.0  35.453272  ...    1.0  0.142088  2.591916   
0 days 23:50:00  0.0    0.0    0.0  35.435031  ...    1.0  0.142088  2.591916   
0 days 23:55:00  0.0    0.0    0.0  35.416767  ...    1.0  0.142088  2.591916   
1 days 00:00:00  0.0    0.0    0.0  34.395198  ...    1.0  0.121889  2.223444   

link                                                                            
id                           V47                         V2                     
properties      status  velocity      flow status  velocity        flow status  
0 days 00:00:00    1.0  0.281040  2.278495    1.0  2.063351  104.552098    1.0  
0 days 00:05:00    1.0  0.281038  2.278476    1.0  2.060275  104.396247    1.0  
0 days 00:10:00    1.0  0.281038  2.278476    1.0  2.057212  104.241037    1.0  
0 days 00:15:00    1.0  0.281038  2.278476    1.0  2.054161  104.086449    1.0  
0 days 00:20:00    1.0  0.281038  2.278476    1.0  2.051123  103.932489    1.0  
...                ...       ...       ...    ...       ...         ...    ...  
0 days 23:40:32    1.0  0.300772  2.438466    1.0  1.430107   72.464989    1.0  
0 days 23:45:00    1.0  0.300769  2.438447    1.0  1.428035   72.359994    1.0  
0 days 23:50:00    1.0  0.300769  2.438447    1.0  1.425879   72.250737    1.0  
0 days 23:55:00    1.0  0.300769  2.438447    1.0  1.423735   72.142085    1.0  
1 days 00:00:00    1.0  0.258012  2.091799    1.0  1.483760   75.183613    1.0  

[314 rows x 45 columns]

In [26]:
df_tanks = net.df_nodes_report['tanks']
df_tanks

id                      T1                              T2            \
properties            head  pressure     demand       head  pressure   
0 days 00:00:00  74.484863  2.984863 -38.823368  65.519113  0.519113   
0 days 00:05:00  74.469805  2.969805 -38.622529  65.538100  0.538100   
0 days 00:10:00  74.454825  2.954825 -38.420952  65.556962  0.556962   
0 days 00:15:00  74.439923  2.939923 -38.220180  65.575698  0.575698   
0 days 00:20:00  74.425100  2.925100 -38.020177  65.594309  0.594309   
...                    ...       ...        ...        ...       ...   
0 days 23:40:32  73.286033  1.786033 -84.947288  67.029611  2.029611   
0 days 23:45:00  73.252966  1.752966 -84.811802  67.018455  2.018455   
0 days 23:50:00  73.219967  1.719967 -84.636796  67.007229  2.007229   
0 days 23:55:00  73.187036  1.687036 -84.462514  66.995931  1.995931   
1 days 00:00:00  73.187036  1.687036 -75.631618  66.995931  1.995931   

id                                  T3                               T4  ...  \
properties          demand        head  pressure     demand        head  ...   
0 days 00:00:00  21.607243  115.942808  3.042808  21.126828  135.021405  ...   
0 days 00:05:00  21.464416  115.985594  3.085594  21.116433  135.042802  ...   
0 days 00:10:00  21.322249  116.028360  3.128360  21.106017  135.064192  ...   
0 days 00:15:00  21.180771  116.071104  3.171104  21.095576  135.085575  ...   
0 days 00:20:00  21.039984  116.113828  3.213828  21.085110  135.106950  ...   
...                    ...         ...       ...        ...         ...  ...   
0 days 23:40:32 -12.531462  116.421810  3.521810  18.219667  135.210602  ...   
0 days 23:45:00 -12.610850  116.458694  3.558694  18.203347  135.225213  ...   
0 days 23:50:00 -12.691510  116.495541  3.595541  18.185106  135.239794  ...   
0 days 23:55:00 -12.771504  116.532351  3.632351  18.166841  135.254346  ...   
1 days 00:00:00  -8.270743  116.532351  3.632351  20.595919  135.254346  ...   

id                                  T5                               T6  \
properties          demand        head  pressure     demand        head   
0 days 00:00:00   7.592518  106.846838  1.046838  17.335417  106.721912   
0 days 00:05:00   7.589998  106.893629  1.093629  17.317890  106.743826   
0 days 00:10:00   7.587309  106.940373  1.140373  17.300512  106.765760   
0 days 00:15:00   7.584689  106.987070  1.187070  17.283149  106.787714   
0 days 00:20:00   7.581997  107.033720  1.233720  17.265803  106.809687   
...                    ...         ...       ...        ...         ...   
0 days 23:40:32   5.191744  107.331792  1.531792  13.161513  107.000000   
0 days 23:45:00   5.182439  107.367299  1.567299  13.141826  107.000000   
0 days 23:50:00   5.172135  107.402755  1.602755  13.122730  107.000000   
0 days 23:55:00   5.161709  107.438160  1.638160  13.103637  107.000000   
1 days 00:00:00  11.026898  107.438160  1.638160  15.914310  107.000000   

id                                           T7                      
properties       pressure    demand        head  pressure    demand  
0 days 00:00:00  5.221912  3.980508  104.543810  2.543810  5.847160  
0 days 00:05:00  5.243826  3.980977  104.585103  2.585103  5.511158  
0 days 00:10:00  5.265760  3.984597  104.626296  2.626296  5.497743  
0 days 00:15:00  5.287714  3.988101  104.667389  2.667389  5.484504  
0 days 00:20:00  5.309687  3.991572  104.708383  2.708383  5.471317  
...                   ...       ...         ...       ...       ...  
0 days 23:40:32  5.500000  0.000000  105.443037  3.443037  3.728009  
0 days 23:45:00  5.500000  0.000000  105.468111  3.468111  3.346553  
0 days 23:50:00  5.500000  0.000000  105.493106  3.493106  3.335915  
0 days 23:55:00  5.500000  0.000000  105.518022  3.518022  3.325380  
1 days 00:00:00  5.500000  0.000000  105.518022  3.518022  9.085998  

[314 rows x 21 columns]

In [30]:
df_pressure = df_tanks.iloc[:, df_tanks.columns.get_level_values(1) == 'pressure']
df_pressure

id,T1,T2,T3,T4,T5,T6,T7
properties,pressure,pressure,pressure,pressure,pressure,pressure,pressure
0 days 00:00:00,2.984863,0.519113,3.042808,2.521405,1.046838,5.221912,2.543810
0 days 00:05:00,2.969805,0.538100,3.085594,2.542802,1.093629,5.243826,2.585103
0 days 00:10:00,2.954825,0.556962,3.128360,2.564192,1.140373,5.265760,2.626296
0 days 00:15:00,2.939923,0.575698,3.171104,2.585575,1.187070,5.287714,2.667389
0 days 00:20:00,2.925100,0.594309,3.213828,2.606950,1.233720,5.309687,2.708383
...,...,...,...,...,...,...,...
0 days 23:40:32,1.786033,2.029611,3.521810,2.710602,1.531792,5.500000,3.443037
0 days 23:45:00,1.752966,2.018455,3.558694,2.725213,1.567299,5.500000,3.468111
0 days 23:50:00,1.719967,2.007229,3.595541,2.739794,1.602755,5.500000,3.493106


In [33]:
df_pressure.to_csv('tanks_pressures.csv', index=False)

In [40]:
df_pressure.subplot()

AttributeError: 'DataFrame' object has no attribute 'subplots'

In [37]:
df_pressure['T5'].min()

properties
pressure    1.046838
dtype: float64